In [4]:
%autosave 30
%load_ext autoreload
%autoreload 2

Autosaving every 30 seconds


In [5]:
cd ..

/home/huawei123/kwx1991442/code-classification


In [6]:
# imports
import os
from tqdm import tqdm
import numpy as np

from datasets import load_dataset
from transformers import (
    AutoTokenizer, GPT2Tokenizer,
    GPT2Config, AutoModelForCausalLM,
    AdamW, get_linear_schedule_with_warmup)

from src.params import (
    PATH_CODEPARROT,
)

path_to_dataset = "./data/the-stack-rust-clean/data/"

2023-08-31 16:27:12.058190: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 16:27:12.873835: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
# load data
dataset = load_dataset(path_to_dataset)

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

In [8]:
# load tokenizer
path_to_tokenizer = PATH_CODEPARROT
start_extra_id: int = 2
stop_etra_id: int = 50

tokenizer = GPT2Tokenizer.from_pretrained(path_to_tokenizer)
# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token
# Add extra tokens to encode whitespaces
extra_tokens = [f"<extratoken_{token_id}>" for token_id in range(start_extra_id, stop_etra_id)]
tokenizer.add_tokens(new_tokens=extra_tokens, special_tokens=True)

48

In [9]:
def encode_whitespaces(text: str):
    """Encode whitespaces with extra tokens"""
    whitespace_tokens_ids = np.arange(start_extra_id, stop_etra_id)
    for token_id in whitespace_tokens_ids[::-1]:
        token = f"<extratoken_{token_id}>"
        text = text.replace(" " * token_id, token)
    return text

a = ''

def tokenize_function(examples):
    texts = examples["content"]
    texts = [encode_whitespaces(text) for text in texts]
    return tokenizer(texts)

In [10]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["content"])

Map (num_proc=4):   0%|          | 0/893792 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (8425 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3778 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3854 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1235 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}

    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result